In [9]:
import statsmodels.api as sm
import pandas as pd
from sklearn.metrics import mean_squared_error
from utils import get_significant_variables
from lags import create_df_with_lags

In [10]:
df = pd.read_csv("../processed_tables/merged_standardized.csv")

## Creación de lags para el precio de la energía

In [11]:
df = create_df_with_lags(df, 3)
df.head()

,precipitacion_amazonas,precipitacion_antioquia,precipitacion_arauca,precipitacion_atlantico,precipitacion_bogota,precipitacion_bolivar,precipitacion_boyaca,precipitacion_caldas,precipitacion_caqueta,precipitacion_casanare,...,temp_VICHADA,temp_avg_pais,temp_avg_represas,brent_value,TRM,Date,energy_price,energy_price_lag_1,energy_price_lag_2,energy_price_lag_3
3,-0.580600,-0.100687,1.718155,-0.183251,-0.287443,3.802742,0.274778,-0.533335,-0.300951,0.339387,...,0.057805,1.150745,0.723472,-0.761153,-1.423606,2021-10-04,286.526116,282.065065,256.940174,216.929726
4,0.016831,1.136810,2.981760,0.521690,-0.560886,0.362265,1.586688,0.365171,0.158697,-0.180160,...,-0.701899,0.112145,-0.197147,-0.614059,-1.411032,2021-10-05,278.157325,286.526116,282.065065,256.940174
5,-0.142496,3.004638,4.689849,-0.306373,-0.567283,0.898967,2.674880,0.219436,2.305734,0.758029,...,1.235347,-0.157838,-0.525688,-0.624360,-1.383612,2021-10-06,261.474788,278.157325,286.526116,282.065065
6,0.132014,0.622153,1.953096,-0.256603,-0.572080,-0.005022,0.895201,-0.454157,-0.524727,-0.042188,...,0.171761,0.802658,0.438749,-0.726955,-1.405736,2021-10-07,235.857848,261.474788,278.157325,286.526116
7,-0.556487,4.243553,2.308684,-0.184229,0.316477,0.581741,0.004270,-0.263669,-0.170919,0.066591,...,0.513628,0.574980,-0.071897,-0.569972,-1.447442,2021-10-08,229.313699,235.857848,261.474788,278.157325


## Partición de la data

Debido a que la naturaleza de la data es de carácter temporal, se realizará una partición de la data en 3 partes: entrenamiento y prueba. La data de test será los últimos 3 meses de la data, es decir desde julio 2023 hasta septiembre 2023.


In [12]:
fecha_corte = "2023-07-01"
X_train = df[df["Date"] < fecha_corte].drop(["Date", "energy_price"], axis=1)
X_test = df[df["Date"] >= fecha_corte].drop(["Date", "energy_price"], axis=1)
y_train = df[df["Date"] < fecha_corte]["energy_price"]
y_test = df[df["Date"] >= fecha_corte]["energy_price"]

## Regresión lineal

Usando `statsmodels` se puede realizar una regresión lineal, para ello se debe importar la librería y crear un modelo de regresión lineal, para luego ajustarlo con los datos de entrenamiento.


In [13]:
X = sm.add_constant(X_train)

# Create a model
model = sm.OLS(y_train, X)

# Fit the model
result = model.fit()
result_summary = result.summary()

# Print out the statistics
print(result_summary)

                            OLS Regression Results                            
Dep. Variable:           energy_price   R-squared:                       0.952
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     135.6
Date:                Sat, 25 Nov 2023   Prob (F-statistic):          2.36e-273
Time:                        15:41:54   Log-Likelihood:                -2785.2
No. Observations:                 549   AIC:                             5712.
Df Residuals:                     478   BIC:                             6018.
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------

In [14]:
get_significant_variables(result_summary)

  Variables significativas
0                    const
1              brent_value
2       energy_price_lag_1
3       energy_price_lag_3


Con esto verificamos que el valor de $R^2$ es de $0.514$, lo cual quiere decir que hay una correlación baja entre las variables exógenas y la variable endógena. Las variables significativas son 15 y están asociados a las temperaturas y precipitaciones de ciertos departamentos y también a la temperatura y precipitación de los departamentos que tienen represas.

In [15]:
X_new_with_intercept = sm.add_constant(X_test)
y_pred = result.predict(X_new_with_intercept)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

print(f"MSE: {mse}")


MSE: 4770.522412238448
